In [73]:
import sys
sys.path.append('scripts')
from lib import *
from sklearn.metrics import confusion_matrix
from math import ceil
import plotly.express as px
from plotly.subplots import make_subplots



In [2]:
dfs, available_classes, available_models = read_raw_reports(os.path.join('..',raw_reports_path))

In [3]:
available_models

['ViT-H-14-378-quickgelu',
 'ViT-B_32',
 'ViT-SO400M-14-SigLIP-384',
 'ViT-L_14',
 'EVA02-E-14-plus',
 'RN101',
 'RN50x64',
 'ViT-bigG-14-CLIPA-336']

In [4]:
available_classes

['ground',
 'concrete',
 'asphalt',
 'concrete plates',
 'grass',
 'paving stones',
 'sett']

In [5]:
all_dfs = []
modelwise_dfs = {}

for class_dict in dfs:
    for model_name in dfs[class_dict]:
        model_df = dfs[class_dict][model_name]
        all_dfs.append(model_df)

        if not model_name in modelwise_dfs:
            modelwise_dfs[model_name] = []

        modelwise_dfs[model_name].append(model_df)


for model_name in modelwise_dfs:
    modelwise_dfs[model_name] = pd.concat(modelwise_dfs[model_name])

In [6]:
merged_df = pd.concat(all_dfs)

In [7]:
true_classes = merged_df.groupby('image_name').agg('first')[class_label].str.replace('_', ' ')

In [8]:
merged_pred = merged_df.groupby('image_name').sum(numeric_only=True)

In [9]:
merged_pred[prediction_label] = merged_pred.idxmax(axis=1)

In [10]:
merged_pred[class_label] = true_classes

In [11]:
merged_pred

,asphalt,concrete,concrete plates,grass,ground,paving stones,sett,prediction,class
image_name,,,,,,,,,
00b7d076504ff9e7b7ed395b1ad8da2f.png,0.480256,0.670878,0.458242,0.857903,4.166124,0.867546,0.499050,ground,ground
013c4d1eb8f0e6d09d3506e69c2677c4.png,0.258615,0.254482,0.076824,0.145211,0.347741,5.989711,0.927621,paving stones,paving stones
0144ebc3606a3de74528b5a0353f8df0.png,0.350016,0.571236,0.223059,0.146684,0.276353,5.372983,1.059888,paving stones,paving stones
01587414f7e8693a05f13f9ba4457798.png,1.008546,0.506683,1.324714,0.168768,0.546040,3.787260,0.657536,paving stones,paving stones
01beabd884a7d4d05af069ec25c096bf.png,0.207833,0.173900,0.211628,0.171041,0.424571,5.749014,1.061423,paving stones,paving stones
...,...,...,...,...,...,...,...,...,...
fef13573ee306eb702444d42b069ea91.png,0.301393,0.144404,0.204715,5.079870,1.436470,0.478786,0.354594,grass,grass
fefe0b8ce34118a31559f138cb1f1c05.png,0.412514,0.165985,0.112019,0.121229,0.228178,5.785807,1.174724,paving stones,sett
fefe69a9ccf153a319a2efa038885ce5.png,0.218333,0.263272,0.269892,0.135827,0.174470,6.424686,0.513515,paving stones,paving stones


In [31]:
true_labels =      merged_pred[class_label]#.tolist()
predicted_labels = merged_pred[prediction_label]#.tolist()

c_mat = confusion_matrix(true_labels,predicted_labels,labels=available_classes, normalize='true') * 100
c_mat = c_mat.round(3)

In [32]:
# sanity check on labels:
print(merged_pred[class_label].unique())

['ground' 'paving stones' 'asphalt' 'sett' 'grass' 'concrete plates'
 'concrete']


In [61]:
fig = px.imshow(c_mat,
                labels=dict(x="Prediction", y="Class"),
                x=available_classes,
                y=available_classes
               , text_auto=True, 
            #    aspect="auto"
                color_continuous_scale=px.colors.sequential.Burg
               )
fig.update_xaxes(side="top")
fig.show()

In [70]:
n_rows = int(ceil(len(available_models)/2))

In [71]:
modelwise_confusion_matrices = {}

for model in modelwise_dfs:
    print(model, modelwise_dfs[model].shape)
    true_labels =      merged_pred[class_label]#.tolist()
    predicted_labels = merged_pred[prediction_label]#.tolist()

    modelwise_confusion_matrices[model] = confusion_matrix(true_labels,predicted_labels,labels=available_classes, normalize='true') * 100
    modelwise_confusion_matrices[model] = modelwise_confusion_matrices[model].round(3)

RN50x64 (729, 10)
ViT-bigG-14-CLIPA-336 (729, 10)
ViT-L_14 (729, 10)
ViT-H-14-378-quickgelu (729, 10)
ViT-SO400M-14-SigLIP-384 (729, 10)
RN101 (729, 10)
ViT-B_32 (729, 10)
EVA02-E-14-plus (729, 10)


In [80]:
fig = make_subplots(rows=n_rows, cols=2)

for i,matrix in enumerate(modelwise_confusion_matrices):
    subplot = px.imshow(modelwise_confusion_matrices[matrix],
                labels=dict(x="Prediction", y="Class"),
                x=available_classes,
                y=available_classes
               , text_auto=True, 
            #    aspect="auto"
                color_continuous_scale=px.colors.sequential.Burg
               )
    
    print(i//2+1,i%2+1)

#     fig.add_trace(subplot, row=i//2+1, col=i%2+1)
    
subplot.show()


1 1
1 2
2 1
2 2
3 1
3 2
4 1
4 2


In [84]:
(modelwise_confusion_matrices['RN101'] - modelwise_confusion_matrices['RN50x64'])


array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])